## (failed) cross-validation model

Read in data

In [3]:
# Your answer goes in here
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

# main data
# matchData = sqlContext.sql("Select * from matchdata3_csv") 

# lookups 
hero_names = sqlContext.sql("Select * from dota_hero_names_csv")
item_ids = sqlContext.sql("Select * from dota_item_ids_csv")
patch_dates = sqlContext.sql("Select * from dota_patch_dates_csv")

In [4]:
# recreate matchData from scratch
matches = sqlContext.sql("Select * from match_csv")
players = sqlContext.sql("Select * from players_csv")
time = sqlContext.sql("Select * from player_time_csv")

from pyspark.sql.functions import *

# join players with match
joined = matches.join(players, ['match_id'])
joined = joined.select(joined.columns[0:51])
#joined = joined.where('account_id > 0')
#joined = joined.withColumn('side', when(joined['player_slot'] < 100, "Radiant").otherwise("Dire"))

#joined.count()

In [5]:
# spread by player_slot & hero_id

temp = joined.select('match_id', 'duration', 'first_blood_time', 'hero_id', 'player_slot', 'radiant_win')
pivot = (temp
         .select('match_id', 'duration', 'first_blood_time', 'hero_id', 'player_slot', 'radiant_win')
         .groupBy('match_id')
         .pivot('player_slot', ['0','1','2','3','4','128','129','130','131','132'])
         .agg(first('hero_id')))

pivot = (pivot
         .withColumnRenamed('0','t0')
         .withColumnRenamed('1','t1')
         .withColumnRenamed('2','t2')
         .withColumnRenamed('3','t3')
         .withColumnRenamed('4','t4')
         .withColumnRenamed('128','t128')
         .withColumnRenamed('129','t129')
         .withColumnRenamed('130','t130')
         .withColumnRenamed('131','t131')
         .withColumnRenamed('132','t132'))

In [6]:
# build dummy columns per side for each hero_id

string = "radiant"
for i in range(114): # 113 heroes
  pivot=pivot.withColumn(string+str(i), when(((col('t0') == i) | (col('t1') == i) | (col('t2') == i) | (col('t3') == i) | (col('t4') == i)), 1)
                         .otherwise(0))

string = "dire"
for i in range(114): # 113 heroes
  pivot=pivot.withColumn(string+str(i), when(((col('t128') == i) | (col('t129') == i) | (col('t130') == i) | (col('t131') == i) | (col('t132') == i)), 1)
                         .otherwise(0))
  
#pivot.count()

In [7]:
# subset important cols
data = matches.select('match_id', 'duration', 'first_blood_time', 'radiant_win')
pivot = pivot.drop('t0','t1','t2','t3','t4','t128','t129','t130','t131','t132')

# join important cols to pivoted/spread data
matchData = data.join(pivot, ['match_id'])

#matchData.count()

In [8]:
# add in data from player_time
matchData = matchData.join(time, ['match_id'])

#matchData.count()

In [9]:
#matchData.groupBy('match_id').count().show()
# since we get data every 60 seconds, the count here represents roughly how long each match lasted

In [10]:
# create sum columns for team gold, xp, lh
modelData = matchData.withColumn("radiant_gold", 
                                 matchData["gold_t_0"] + matchData["gold_t_1"] + matchData["gold_t_2"] + matchData["gold_t_3"] + matchData["gold_t_4"])
modelData = modelData.withColumn("radiant_xp", 
                                 matchData["xp_t_0"] + matchData["xp_t_1"] + matchData["xp_t_2"] + matchData["xp_t_3"] + matchData["xp_t_4"])
modelData = modelData.withColumn("radiant_lh", 
                                 matchData["lh_t_0"] + matchData["lh_t_1"] + matchData["lh_t_2"] + matchData["lh_t_3"] + matchData["lh_t_4"])

modelData = modelData.withColumn("dire_gold", 
                                 matchData["gold_t_128"] + matchData["gold_t_129"] + matchData["gold_t_130"] + matchData["gold_t_131"] + matchData["gold_t_132"])
modelData = modelData.withColumn("dire_xp", 
                                 matchData["xp_t_128"] + matchData["xp_t_129"] + matchData["xp_t_130"] + matchData["xp_t_131"] + matchData["xp_t_132"])
modelData = modelData.withColumn("dire_lh", 
                                 matchData["lh_t_128"] + matchData["lh_t_129"] + matchData["lh_t_130"] + matchData["lh_t_131"] + matchData["lh_t_132"])

In [11]:
# create differential columns for gold, xp, lh
modelData = modelData.withColumn("golddiff", modelData["radiant_gold"] - modelData["dire_gold"])
modelData = modelData.withColumn("xpdiff", modelData["radiant_xp"] - modelData["dire_xp"])
modelData = modelData.withColumn("lhdiff", modelData["radiant_lh"] - modelData["dire_lh"])

In [12]:
# recode 'times' to float
modelData = modelData.withColumn('times', modelData.times.cast('float'))
#modelData.dtypes

In [13]:
# drop unnecessary / old columns
modelData = modelData.drop("match_id","gold_t_0", "gold_t_1", "gold_t_2", "gold_t_3", "gold_t_4", 
                           "xp_t_0", "xp_t_1", "xp_t_2", "xp_t_3", "xp_t_4",
                           "lh_t_0", "lh_t_1", "lh_t_2", "lh_t_3", "lh_t_4",
                           "gold_t_128", "gold_t_129", "gold_t_130", "gold_t_131", "gold_t_132",
                           "xp_t_128", "xp_t_129", "xp_t_130", "xp_t_131", "xp_t_132",
                           "lh_t_128", "lh_t_129", "lh_t_130", "lh_t_131", "lh_t_132")

In [14]:
#print(modelData.columns)

## Inputs
* Heroes (dummy variables)
* per-minute gold (team sum)
* per-minute xp (team sum)
* per-minute last-hits (team sum)
* final result (coded as 'radiant win')

Logistic regression predicting (radiant-side) win probability given a timestamp and gold, xp, lh
* this means we may have to subset by timestamp(?)

In [16]:
# define dependent variable
target = 'radiant_win'

In [17]:
# recode 'radiant_win' into 0/1
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf # UDF - user defined function
from pyspark.sql.functions import *

# create user defined function that codes as 1 if "CDK" and 0 otherwise; cast as type integer
get_01_label = udf(lambda x: 1 if x == 'True' else 0, IntegerType())

# withColumn("new_name", function(old_df["old_name"]))
modelData = modelData.withColumn("radiant_win", get_01_label(modelData["radiant_win"]))


In [18]:
# step 1 to identify premade onehot encoded heroes (they start with 'radiant' or 'dire')
rhero_input = [col for col in modelData.columns if ('radiant') in col]
dhero_input = [col for col in modelData.columns if ('dire') in col]

hero_input = rhero_input + dhero_input
#hero_input

In [19]:
# identify the non-hero columns (they start with 'radiant_' or 'dire_')
rnonhero_input = [col for col in modelData.columns if ('radiant_') in col]
dnonhero_input = [col for col in modelData.columns if ('dire_') in col]

nonhero_input = rnonhero_input + dnonhero_input
#nonhero_input

In [20]:
# select only premade onehot encoded heroes
hero_input = list(set(hero_input)-set(nonhero_input))
#hero_input

In [21]:
### no categorical variables because we've already one-hot encoded heroes
#dtypes = modelData.dtypes
#cat_input = []
#for i in range(0, len(modelData.columns)):
#   if dtypes[i][1] == 'string':      # if data type is string
#      cat_input.append(dtypes[i][0])  # append to list of categorical variables
#cat_input = list(set(cat_input)-set(target))   # returns elements that exist in list A that are NOT in list B (remove dependent variable from list)
   
#cat_input

In [22]:
# numerical variables
num_input = list(set(modelData.columns) - set([target]) - set(hero_input))
#num_input

####Begin building pipeline
Numeric/Continuous variables

In [24]:
# impute missing values
from pyspark.ml.feature import Imputer

numImputer = Imputer(strategy='median', inputCols=num_input, outputCols=num_input)  # default is mean imputation
#test = numImputer.fit(modelData).transform(modelData)
#test.show()

In [25]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

numAssembler = VectorAssembler(inputCols= num_input, outputCol="features")
#test = numAssembler.transform(modelData)
#test.select("features").show()

In [26]:
### run scaler to transform all columns in num_input
#minmax = [MinMaxStandardizer( inputCol = column, outputCol = column+"_standardized") for column in num_input]

from pyspark.ml.feature import MinMaxScaler

minmax = MinMaxScaler(inputCol = "features", outputCol = "scaled") 
#test = minmax.fit(test).transform(test)
#test.select("features","scaled").show()
#test.select("features","scaled").head()

Add pre-coded categorical variables

In [28]:
input_cols = ["scaled"]
for i in hero_input:
  input_cols.append(i)
#input_cols

In [29]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

finalAssembler = VectorAssembler(inputCols= input_cols, outputCol="finalFeatures")
#test=finalAssembler.transform(test)
#test.select("features","finalFeatures").head()

In [30]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol = "finalFeatures", 
                                    labelCol = target , 
                                    maxIter=10)

In [31]:
### staging area

stages = []
stages.append(numImputer)
stages.append(numAssembler)
stages.append(minmax)
stages.append(finalAssembler)
stages.append(lr) # could run this post-regression

stages

In [32]:
### Set up the ML Pipeline
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=stages)  # with logit
pipeline2 = Pipeline(stages=[numImputer, numAssembler, minmax, finalAssembler]) # separate logit from data transforms

In [33]:
### break the pipeline between data setup and regression - data setup

pipedData = pipeline2.fit(modelData).transform(modelData)
#pipedData.show()

In [34]:
### convert to RDD to train/test split
rdd = pipedData.rdd

# Split data into training (10%) and test (90%)
train, test = rdd.randomSplit([0.1, 0.9])

In [35]:
trainDF = train.toDF()
testDF = test.toDF()

In [36]:
### break the pipeline between data setup and regression - regression part
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# We use a ParamGridBuilder to construct a grid of parameters to search over.
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]) \
    .build()
    
    
crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(labelCol=target), 
                          numFolds=2)  # 2-fold cross-validation

# run the crossvalidation
cvModel = crossval.fit(trainDF)


In [37]:
bestModel = cvModel.bestModel
bestParams = bestModel.extractParamMap()
bestParams
# regParam = .75
# elasticNetParam = .25

In [38]:
from pyspark.ml.classification import LogisticRegression

bestModel = LogisticRegression(featuresCol = "finalFeatures", 
                                    labelCol = target , 
                                    maxIter=10,
                                    regParam = 0.25,
                                    elasticNetParam = 0.75)

In [39]:
#val lrmodel = bestModel.asInstanceOf[LogisticRegressionModel]
bestModel.fit(trainDF).coefficients #or weights?
bestModel.fit(trainDF).intercept

In [40]:
trainResults = bestModel.fit(trainDF).transform(trainDF)
testResults = bestModel.fit(testDF).transform(testDF)

In [41]:
print(trainResults.columns)

In [42]:
# classification evaluation metrics - training set
eval = trainResults.select(target,'prediction')

TP = eval[(eval[target] == 1) & (eval['prediction'] == 1)].count()
FP = eval[(eval[target] == 0) & (eval['prediction'] == 1)].count()
TN = eval[(eval[target] == 0) & (eval['prediction'] == 0)].count()
FN = eval[(eval[target] == 1) & (eval['prediction'] == 0)].count()

print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)
print("Total:", eval.count())

# accuracy rate, false positive rate, false negative rate
A = float(TP + TN) / (TP + TN + FP + FN)
print("accuracy:", A)

P = float(TP) / (TP + FP)
print("precision:", P)

R = float(TP) / (TP + FN)
print("recall(TPR):", R)

S = float(TN) / (TN + FP)
print("specificity(TNR):", S)

N = float(FN) / (TP + FN)
print("FNR:", N)

F = float(FP) / (FP + TN)
print("FPR:", F)

In [43]:
# classification evaluation metrics - testing set
eval = testResults.select(target,'prediction')

TP = eval[(eval[target] == 1) & (eval['prediction'] == 1)].count()
FP = eval[(eval[target] == 0) & (eval['prediction'] == 1)].count()
TN = eval[(eval[target] == 0) & (eval['prediction'] == 0)].count()
FN = eval[(eval[target] == 1) & (eval['prediction'] == 0)].count()

print("True Positives:", TP)
print("False Positives:", FP)
print("True Negatives:", TN)
print("False Negatives:", FN)
print("Total:", eval.count())

# accuracy rate, false positive rate, false negative rate
A = float(TP + TN) / (TP + TN + FP + FN)
print("accuracy:", A)

P = float(TP) / (TP + FP)
print("precision:", P)

R = float(TP) / (TP + FN)
print("recall(TPR):", R)

S = float(TN) / (TN + FP)
print("specificity(TNR):", S)

N = float(FN) / (TP + FN)
print("FNR:", N)

F = float(FP) / (FP + TN)
print("FPR:", F)

In [44]:
# calculate the fpr and tpr for all thresholds of the classification
import numpy as np
import pandas as pd
from sklearn import metrics

rocauc = eval.toPandas()

fpr, tpr, thresholds = metrics.roc_curve(rocauc[target], rocauc['prediction'])
roc_auc = metrics.auc(fpr, tpr)

# ROC
import matplotlib.pyplot as plt

f, ax = plt.subplots(figsize=(5, 5))

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
#plt.show()
display(f)
